## Fase 6: Evaluación y Métricas

### Objetivo
Evaluar el rendimiento del modelo de recuperación de información mediante la aplicación de métricas estándar como recall, precision y F1-score sobre un conjunto de consultas y sus resultados recuperados.

### Descripción
En esta fase, se implementa la evaluación del modelo utilizando un conjunto de consultas previamente definidas y sus respectivas categorías esperadas. Se aplican las métricas de recall, precision y F1-score para evaluar qué tan bien el modelo recupera documentos relevantes en comparación con las categorías esperadas para cada consulta.

### Pasos

1. **Carga de Consultas**: 
   - Se lee un archivo de texto que contiene las consultas junto con sus categorías esperadas.
   - Cada consulta se almacena como una tupla que contiene el nombre de la consulta y un conjunto de números que representan las categorías esperadas.

2. **Recuperación de Documentos**:
   - Para cada consulta, se utiliza un modelo de recuperación de información para obtener una lista de documentos ordenados según su relevancia para la consulta.

3. **Cálculo de Métricas**:
   - Se calcula el recall, precision y F1-score para cada consulta, comparando los documentos recuperados con las categorías esperadas.
   - El recall se calcula como la proporción de documentos relevantes recuperados respecto al total de documentos relevantes.
   - La precision se calcula como la proporción de documentos relevantes recuperados respecto al total de documentos recuperados.
   - El F1-score se calcula como la media armónica de recall y precision, proporcionando una medida combinada de ambas métricas.

4. **Promedio de Métricas**:
   - Se calcula el promedio de recall, precision y F1-score sobre todas las consultas evaluadas para obtener una evaluación general del modelo.

5. **Matriz de Confusión**:
   - Se genera una matriz de confusión para cada consulta, mostrando la distribución de documentos relevantes recuperados correctamente y aquellos que fueron incorrectamente recuperados.

6. **Resultados y Evaluación**:
   - Se imprimen los resultados de recall, precision y F1-score para cada consulta.
   - Se imprime el promedio de estas métricas para evaluar el rendimiento general del modelo en la recuperación de información.

Esta fase es crucial para validar la efectividad del modelo en la tarea de recuperación de información, proporcionando insights sobre qué tan bien se comporta frente a las expectativas definidas por las consltas y sus categorías esperadas.


In [1]:
! pip install ipynb --upgrade

In [18]:
from ipynb.fs.full.SRI_Reuters_converted import buscar_documentos

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Ãƒndice invertido guardado en inverted_index.txt
Busqueda para la query: lin-oil
Resultados ordenados (nombre y distancia):
Nombre: 6.txt, Distancia: 0.06864310990282788

Nombres de documentos ordenados:
6.txt
Busqueda para la query: lin-oil
Resultados ordenados (nombre y distancia):
Nombre: 6.txt, Distancia: 0.15617376188860607

Nombres de documentos ordenados:
6.txt


In [19]:
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [20]:
def leer_categorias(file_path):
    # Lista para almacenar las consultas y sus categorías
    queries = []
    
    # Abrir el archivo de texto
    with open(file_path, 'r') as file:
        # Iterar sobre cada línea en el archivo
        for line in file:
            line = line.strip()
            if ': ' in line:
                # Separar el nombre de la consulta y los números de categoría
                query_name, numbers_str = line.split(': ', 1)
                # Convertir los números de categoría a un conjunto de enteros
                numbers = set(map(int, re.findall(r'\d+', numbers_str)))  # Convertir a conjunto directamente
                # Agregar la consulta y sus categorías a la lista
                queries.append((query_name, numbers))
    
    return queries

In [21]:
def calculo_recall(predichos, gt):
    # Calcular verdaderos positivos (TP) como la longitud de la intersección
    TP = len(predichos & gt)  # Intersección
    # Calcular falsos negativos (FN) como la longitud de la diferencia
    FN = len(gt - predichos)  # Diferencia
    
    # Calcular el total de verdaderos positivos más falsos negativos
    total = TP + FN
    
    # Evitar división por cero
    if total == 0:
        return 0.0
    
    # Calcular el recall como TP dividido por el total
    recall = TP / total
    return recall

In [22]:
def calculo_precision(predichos, gt):
    # Calcular verdaderos positivos (TP) como la longitud de la intersección
    TP = len(predichos & gt)  # Intersección
    # Calcular falsos positivos (FP) como la longitud de la diferencia
    FP = len(predichos - gt)  # Diferencia
    
    # Calcular el total de verdaderos positivos más falsos positivos
    total = TP + FP
    
    # Evitar división por cero
    if total == 0:
        return 0.0
    
    # Calcular la precisión como TP dividido por el total
    precision = TP / total
    return precision

In [23]:
def calculo_f1_score(precision, recall):
    # Calcular el F1-score utilizando la fórmula estándar
    if precision + recall == 0:
        return 0.0  # Evitar división por cero
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

In [24]:
file_path = 'inverted_index.txt'
queries = leer_categorias(file_path)

total_recall = 0
total_precision = 0
total_f1_score = 0
total_queries = len(queries)

vectorizerBoW = 'BoW'
vectorizerTF_IDF = 'TF-IDF'

# Bag of Words

In [25]:
for query_name, numbers_set in queries:
    query = query_name  # Assuming query_name holds the query
    resultados_ordenados, nombres_ordenados = buscar_documentos(query, vectorizerBoW)  # Obtener los resultados ordenados
    
    print("====================================================================")
    print("Resultados para: ", query_name)
    print("Documentos recuperados: ", nombres_ordenados)
    print("Cantidad de predicciones: ", len(nombres_ordenados))
    print("Números esperados: ", numbers_set)
    
    # Convertir nombres_ordenados a enteros
    nombres_ordenados_nums = set(map(int, [nombre.split('.')[0] for nombre in nombres_ordenados]))
    
    # Calcular Recall y Precision usando solo los números de los documentos
    recall = calculo_recall(nombres_ordenados_nums, numbers_set)
    precision = calculo_precision(nombres_ordenados_nums, numbers_set)
    f1_score = calculo_f1_score(precision, recall)
    
    total_recall += recall
    total_precision += precision
    total_f1_score += f1_score
    
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1-Score: ", f1_score)

Resultados para:  cocoa
Documentos recuperados:  ['10471.txt', '10491.txt', '8326.txt', '3190.txt', '11811.txt', '12813.txt', '8850.txt', '3225.txt', '10619.txt', '10506.txt', '10613.txt', '8978.txt', '9903.txt', '14418.txt', '5880.txt', '5491.txt', '7367.txt', '10449.txt', '11462.txt', '10742.txt', '6414.txt', '10586.txt', '11341.txt', '10014.txt', '3310.txt', '7311.txt', '10584.txt', '9450.txt', '6407.txt', '4147.txt', '13271.txt', '12763.txt', '275.txt', '5598.txt', '6128.txt', '9559.txt', '10505.txt', '10122.txt', '5382.txt', '5168.txt', '5192.txt', '6493.txt', '9953.txt', '11843.txt', '11459.txt', '10403.txt', '12340.txt', '1.txt', '7071.txt', '1889.txt', '8961.txt', '4470.txt', '12355.txt', '13462.txt', '4564.txt', '10760.txt', '5258.txt', '2521.txt', '11224.txt']
Cantidad de predicciones:  59
Números esperados:  {1, 8961, 6405, 5382, 6407, 10760, 10505, 5258, 10506, 10122, 12813, 6414, 7311, 5880, 8850, 8978, 275, 13462, 3225, 10014, 7071, 10403, 8326, 11811, 10491, 5168, 9903, 

In [26]:
# Calcular promedios si es necesario
average_recall = total_recall / total_queries
average_precision = total_precision / total_queries
average_f1_score = total_f1_score / total_queries

print("Recall promedio: ", average_recall)
print("Precision promedio: ", average_precision)
print("F1-Score promedio: ", average_f1_score)

Recall promedio:  0.47618672819411945
Precision promedio:  0.338086807194287
F1-Score promedio:  0.3423767160812758


# TF-IDF

In [27]:
for query_name, numbers_set in queries:
    query = query_name  # Assuming query_name holds the query
    resultados_ordenados, nombres_ordenados = buscar_documentos(query, vectorizerTF_IDF)  # Obtener los resultados ordenados
    
    print("====================================================================")
    print("Resultados para: ", query_name)
    print("Documentos recuperados: ", nombres_ordenados)
    print("Cantidad de predicciones: ", len(nombres_ordenados))
    print("Números esperados: ", numbers_set)
    
    # Convertir nombres_ordenados a enteros
    nombres_ordenados_nums = set(map(int, [nombre.split('.')[0] for nombre in nombres_ordenados]))
    
    # Calcular Recall y Precision usando solo los números de los documentos
    recall = calculo_recall(nombres_ordenados_nums, numbers_set)
    precision = calculo_precision(nombres_ordenados_nums, numbers_set)
    f1_score = calculo_f1_score(precision, recall)
    
    total_recall += recall
    total_precision += precision
    total_f1_score += f1_score
    
    print("Recall: ", recall)
    print("Precision: ", precision)
    print("F1-Score: ", f1_score)

Resultados para:  cocoa
Documentos recuperados:  ['10505.txt', '5258.txt', '10586.txt', '10506.txt', '9953.txt', '10760.txt', '9450.txt', '13271.txt', '9903.txt', '12813.txt', '9559.txt', '5382.txt', '10471.txt', '10491.txt', '10584.txt', '8326.txt', '11843.txt', '11811.txt', '8850.txt', '8961.txt', '5168.txt', '3190.txt', '7071.txt', '3225.txt', '5491.txt', '13462.txt', '5192.txt', '6128.txt', '10619.txt', '8978.txt', '10403.txt', '6493.txt', '275.txt', '10742.txt', '1.txt', '5598.txt', '5880.txt', '10122.txt', '10613.txt', '10449.txt', '11462.txt', '4147.txt', '6407.txt', '6414.txt', '12763.txt', '10014.txt', '11224.txt', '7367.txt', '11459.txt', '7311.txt', '3310.txt', '2521.txt', '1889.txt', '4470.txt', '14418.txt', '12340.txt', '11341.txt', '12355.txt', '4564.txt']
Cantidad de predicciones:  59
Números esperados:  {1, 8961, 6405, 5382, 6407, 10760, 10505, 5258, 10506, 10122, 12813, 6414, 7311, 5880, 8850, 8978, 275, 13462, 3225, 10014, 7071, 10403, 8326, 11811, 10491, 5168, 9903, 

In [28]:
# Calcular promedios si es necesario
average_recall = total_recall / total_queries
average_precision = total_precision / total_queries
average_f1_score = total_f1_score / total_queries

print("Recall promedio: ", average_recall)
print("Precision promedio: ", average_precision)
print("F1-Score promedio: ", average_f1_score)

Recall promedio:  0.9523734563882387
Precision promedio:  0.676173614388574
F1-Score promedio:  0.6847534321625515
